# Load Library

In [ ]:
!pip install folium

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium import plugins

%matplotlib inline

# Load Data

In [ ]:
!wget https://www.dropbox.com/s/8xn2hutam8ozk30/simple_person.csv?dl=0 -O simple_person.csv
!wget https://www.dropbox.com/s/1pm1dzzu1js1loe/th_province_location.csv?dl=0 -O th_province_location.csv

In [ ]:
person_df = pd.read_csv('simple_person.csv')
person_df.head(3)

In [ ]:
location_df = pd.read_csv('th_province_location.csv')
location_df.head(5)

In [ ]:
df = person_df.merge(location_df, on = 'province', how = 'left')
df.head(2)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

# Count per Province

In [ ]:
count_df = df.groupby(['province','lat','long']).size().reset_index(name='count')
count_df

# Scatter plot

In [ ]:
count_df.plot(kind='scatter', x='long', y='lat', alpha=0.6)
plt.show()

In [ ]:
count_df.plot(kind='scatter', x='long', y='lat', s=count_df['count'], alpha=0.6)
plt.show()

In [ ]:
count_df.plot(
    kind='scatter', 
    x='long', 
    y='lat',
    s=count_df['count']*2,
    label='count',
    c='count',
    cmap=plt.get_cmap("jet"),
    colorbar=True,
    alpha=0.4, 
    figsize=(15,8)
)
plt.legend()
plt.show()

# Geolocation Plot

## Default Location

In [ ]:
bangkok_location = location_df.set_index('province').loc['กรุงเทพมหานคร'].values
bangkok_location

## Create Map

In [ ]:
def create_map(default_zoom_start = 7, tile=None):
    if tile is None:
        base_map = folium.Map(
            location = bangkok_location, 
            control_scale = True, 
            zoom_start = default_zoom_start
        )
    else:
        base_map = folium.Map(
            location = bangkok_location, 
            control_scale = True, 
            zoom_start = default_zoom_start,
            tiles=tile,
        )
    return base_map

In [ ]:
# default_zoom_start = 5, 10, 15
base_map = create_map(default_zoom_start = 15)
base_map

In [ ]:
base_map = create_map(default_zoom_start = 15, tile = 'Stamen Toner')
base_map

In [ ]:
base_map = create_map(default_zoom_start = 15, tile = 'Stamen Terrain')
base_map

## Add Icon,Marker

In [ ]:
# prefix = icon folder (set)
base_map = create_map()
folium.Marker(
    bangkok_location + 1,
    popup='Test1',
    icon=folium.Icon(color='green')
).add_to(base_map)

folium.Marker(
    bangkok_location,
    popup='Test2',
    icon=folium.Icon(color='red',icon='university', prefix='fa')
).add_to(base_map)

folium.Marker(
    bangkok_location - 1,
    popup='Test3',
    icon=folium.Icon(color='blue',icon='bar-chart', prefix='fa')
).add_to(base_map)

base_map

In [ ]:
# circle in meters
base_map = create_map()
folium.Circle(
    radius = 100000,
    location = bangkok_location,
    popup = 'Test1'
).add_to(base_map)

folium.Circle(
    radius = 50000,
    location = bangkok_location+2,
    popup = 'Test2',
    color = '#3186cc',
    fill = True,
    fill_color = '#3186cc'
).add_to(base_map)

base_map

## Plot Each Province

In [ ]:
for index, row in count_df.head(5).iterrows():
    province = row['province']
    lat = row['lat']
    long = row['long']
    count = row['count']
    print(province, lat, long, count)

In [ ]:
base_map = create_map()

for index, row in count_df.iterrows():
    province = row['province']
    lat = row['lat']
    long = row['long']
    count = row['count']
    
    folium.Circle(
        radius = count * 50,
        location = [lat, long], 
        popup = '{0}'.format(count),
        color = '#3186cc',
        fill = True,
        fill_color = '#3186cc'
    ).add_to(base_map)
    
base_map

In [ ]:
count_df['marker_color'] = pd.cut(
    count_df['count'], 
    bins = 4, 
    labels = ['red', 'orange', 'yellow', 'green']
)
count_df

In [ ]:
base_map = create_map()

for index, row in count_df.iterrows():
    province = row['province']
    lat = row['lat']
    long = row['long']
    count = row['count']
    marker_color = row['marker_color']
    print(row)
    
    folium.Circle(
        radius = count * 50,
        location = [lat, long],
        popup = '{0}'.format(count),
        color = marker_color,
        fill = True,
        fill_color = marker_color
    ).add_to(base_map)
    
base_map

# USA Plot (Unemployment)

In [ ]:
# Automatic color
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(m)

folium.LayerControl().add_to(m)
m

In [ ]:
# user-defined color bin
bins = list(state_data['Unemployment'].quantile([0, 0.25, 0.5, 0.75, 1]))
m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Unemployment Rate (%)',
    bins=bins,
    reset=True
).add_to(m)
m

In [ ]:
# TOPO-JSON Style1
# m = folium.Map vs folium.TopoJson
# topo-json: we can adjust style in details
import branca
import json
import requests

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
county_data = f'{url}/us_county_data.csv'
county_geo = f'{url}/us_counties_20m_topo.json'

df = pd.read_csv(county_data, na_values=[' '])

colorscale = branca.colormap.linear.YlOrRd_09.scale(0, 50e3)
employed_series = df.set_index('FIPS_Code')['Employed_2011']

def style_function(feature):
    employed = employed_series.get(int(feature['id'][-5:]), None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if employed is None else colorscale(employed)
    }

m = folium.Map(
    location=[48, -102],
    tiles='cartodbpositron',
    zoom_start=3
)

folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function
).add_to(m)
m

In [ ]:
# TOPO-JSON Style2
colorscale = branca.colormap.linear.YlGnBu_09.scale(0, 30)
employed_series = df.set_index('FIPS_Code')['Unemployment_rate_2011']

def style_function(feature):
    employed = employed_series.get(int(feature['id'][-5:]), None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if employed is None else colorscale(employed)
    }


m = folium.Map(
    location=[48, -102],
    tiles='cartodbpositron',
    zoom_start=3
)

folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function
).add_to(m)
m

In [ ]:
# TOPO-JSON Style3
colorscale = branca.colormap.linear.PuRd_09.scale(0, 100000)
employed_series = df.set_index('FIPS_Code')['Median_Household_Income_2011'].dropna()

def style_function(feature):
    employed = employed_series.get(int(feature['id'][-5:]), None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if employed is None else colorscale(employed)
    }

m = folium.Map(
    location=[48, -102],
    tiles='cartodbpositron',
    zoom_start=3
)

folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function
).add_to(m)
m

**Reference** : https://python-visualization.github.io/folium/quickstart.html